# Experiance Analysis

In [21]:
import pandas as pd
import numpy as np
import matplotlib
from sqlalchemy import create_engine
from urllib.parse import quote

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

### Connecting to the database

In [2]:
username = 'postgres'
password = 'nati@postgres'
hostname = 'localhost'
port = '5432'
database_name = 'TellCo'

# Escape the special characters in the password
escaped_password = quote(password, safe='')

# Create the database engine
engine = create_engine(f'postgresql://{username}:{escaped_password}@{hostname}:{port}/{database_name}')

# Establish a connection
with engine.connect() as connection:
    # Query the data and load it into a pandas DataFrame
    query = """
        SELECT *
        FROM xdr_data
    """
    df = pd.read_sql(query, connection)

df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [18]:
selected_fields = ['MSISDN/Number', 'TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)']
duplicates = df[df.duplicated(subset=selected_fields, keep=False)].copy()
duplicates['Count'] = duplicates.groupby(selected_fields)['MSISDN/Number'].transform('count')
duplicates.loc[:, ['MSISDN/Number', 'TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)', 'Count']]

,MSISDN/Number,TCP DL Retrans. Vol (Bytes),Avg RTT DL (ms),Avg Bearer TP DL (kbps),Count
2,4.188282e+10,2.080991e+07,109.795706,6.000000,1036
3,4.188282e+10,2.080991e+07,109.795706,44.000000,179
10,3.366565e+10,2.080991e+07,109.795706,6.000000,2
19,4.188282e+10,2.080991e+07,109.795706,6.000000,1036
25,4.188282e+10,2.080991e+07,63.000000,22.000000,4
...,...,...,...,...,...
149963,4.188282e+10,2.080991e+07,69.000000,13300.045927,10
149985,4.188282e+10,2.080991e+07,109.795706,60.000000,74
149991,4.188282e+10,2.080991e+07,27.000000,62.000000,14
149992,4.188282e+10,2.080991e+07,37.000000,23.000000,46


In [12]:
df.columns

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [19]:
def detect_outliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)))
    return outliers

In [20]:
# Iterate over the columns and identify outliers and missing values
for column in df.columns:
    # Fetch data from the DataFrame for the specified column
    data = df[column]

    # Check if the column is numeric
    if np.issubdtype(data.dtype, np.number):
        # Identify outliers
        outliers = detect_outliers(data)

        # Calculate the mean of the column
        column_mean = data.mean()

        # Replace outliers and missing values with the mean
        df.loc[outliers, column] = column_mean
        df.loc[data.isnull(), column] = column_mean
    else:
        # Calculate the mode of the column
        column_mode = data.mode().values[0]
        
        # Replace missing values with the mode
        df.loc[data.isnull(), column] = column_mode

### Experiance and Engagment Score for each user

#### Experiance Score

Let's calculate each users' experiance score.

In [45]:
# Create a copy of the DataFrame to store results without modifying the original
result_df = df.copy()

# Select relevant columns for experience metrics
experience_metrics = result_df[['TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)']]

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
normalized_experience_metrics = pd.DataFrame(scaler.fit_transform(experience_metrics), columns=experience_metrics.columns)

# Perform k-means clustering with k=3 on normalized data
kmeans = KMeans(n_clusters=3, random_state=42)
result_df['Experience Cluster'] = kmeans.fit_predict(normalized_experience_metrics)

# Identify the samples belonging to the worst experience cluster
worst_experience_cluster_label = np.argmax(kmeans.transform(normalized_experience_metrics), axis=1)
worst_experience_cluster_samples = normalized_experience_metrics.iloc[worst_experience_cluster_label == worst_experience_cluster_label.max()]

# Calculate the centroid of the worst experience cluster
worst_experience_centroid = worst_experience_cluster_samples.mean()

# Calculate the experience score as Euclidean distance to the worst experience cluster centroid
result_df['Experience Score'] = pairwise_distances_argmin_min(normalized_experience_metrics, worst_experience_centroid.values.reshape(1, -1))[1]

# Display the results
print("User Data with Experience Cluster and Experience Score:")
result_df[['TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)', 'Experience Cluster', 'Experience Score']].head(20)

c:\Users\User\miniconda3\envs\telecom_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


User Data with Experience Cluster and Experience Score:


,TCP DL Retrans. Vol (Bytes),Avg RTT DL (ms),Avg Bearer TP DL (kbps),Experience Cluster,Experience Score
0,2.080991e+07,42.000000,23.0,2,1.080634
1,2.080991e+07,65.000000,16.0,2,1.018734
2,2.080991e+07,109.795706,6.0,0,0.986373
3,2.080991e+07,109.795706,44.0,0,0.985932
4,2.080991e+07,109.795706,6.0,0,0.986373
5,2.080991e+07,109.795706,70.0,0,0.985631
6,2.080991e+07,102.000000,22.0,0,0.982860
7,1.952000e+04,39.000000,3698.0,1,0.552276
8,2.080991e+07,109.795706,46.0,0,0.985909
9,3.231397e+06,97.000000,3845.0,1,0.272044


#### Engagment Score

Let's calculate engagment score for each user.

In [47]:
# Create a copy of the DataFrame to store results without modifying the original
result_df_engagement = df.copy()

# 1. Correct the column name for session duration
result_df_engagement = result_df_engagement.rename(columns={'Dur. (ms)': 'Session Duration (ms)'})

# 2. Calculate session frequency for each user
session_frequency = result_df_engagement.groupby('MSISDN/Number').size().reset_index(name='Session Frequency')

# Merge session frequency back into the DataFrame
result_df_engagement = pd.merge(result_df_engagement, session_frequency, on='MSISDN/Number', how='left')

# 3. Create 'Total UL + DL (Bytes)' column
result_df_engagement['Total UL + DL (Bytes)'] = result_df_engagement['Total UL (Bytes)'] + result_df_engagement['Total DL (Bytes)']

# Select relevant columns for engagement metrics
engagement_metrics = result_df_engagement[['Session Duration (ms)', 'Total UL + DL (Bytes)', 'Session Frequency']]

# Normalize the data using Min-Max scaling
scaler_engagement = MinMaxScaler()
normalized_engagement_metrics = pd.DataFrame(scaler_engagement.fit_transform(engagement_metrics), columns=engagement_metrics.columns)

# Perform k-means clustering with k=3 on normalized data
kmeans_engagement = KMeans(n_clusters=3, random_state=42)
result_df_engagement['Engagement Cluster'] = kmeans_engagement.fit_predict(normalized_engagement_metrics)

# Identify the samples belonging to the least engaged cluster
least_engaged_cluster_label = np.argmin(kmeans_engagement.transform(normalized_engagement_metrics), axis=1)
least_engaged_cluster_samples = normalized_engagement_metrics.iloc[least_engaged_cluster_label == least_engaged_cluster_label.min()]

# Calculate the centroid of the least engaged cluster
least_engaged_centroid = least_engaged_cluster_samples.mean()

# Calculate the engagement score as Euclidean distance to the least engaged cluster centroid
result_df_engagement['Engagement Score'] = pairwise_distances_argmin_min(normalized_engagement_metrics, least_engaged_centroid.values.reshape(1, -1))[1]

# Display the results
print("User Data with Engagement Cluster and Engagement Score:")
result_df_engagement[['Session Duration (ms)', 'Total UL + DL (Bytes)', 'Session Frequency', 'Engagement Cluster', 'Engagement Score']].head(20)

c:\Users\User\miniconda3\envs\telecom_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


User Data with Engagement Cluster and Engagement Score:


,Session Duration (ms),Total UL + DL (Bytes),Session Frequency,Engagement Cluster,Engagement Score
0,104608.560347,345629377.0,2,0,0.101995
1,104608.560347,707185356.0,2,2,0.465379
2,104608.560347,307690973.0,25196,1,1.003227
3,104608.560347,889352748.0,25196,1,1.198533
4,104608.560347,607681403.0,1,2,0.359548
5,104608.560347,784759966.0,3,2,0.548399
6,104608.560347,118487293.0,1,0,0.193915
7,104608.560347,834163359.0,25196,1,1.166887
8,104608.560347,903975407.0,1,2,0.676446
9,104608.560347,872988322.0,8,2,0.643124


#### Satisfaction Score

In [51]:
result_df_combined = result_df.copy()

# Add 'Engagement Score' to result_df_combined
result_df_combined['Engagement Score'] = result_df_engagement['Engagement Score']

# Calculate the average satisfaction score
result_df_combined['Satisfaction Score'] = (result_df_combined['Experience Score'] + result_df_combined['Engagement Score']) / 2

# Identify the top 10 satisfied customers
top_satisfied_customers = result_df_combined.sort_values(by='Satisfaction Score', ascending=False).head(10)

# Display the results
print("Top 10 Satisfied Customers:")
top_satisfied_customers[['MSISDN/Number', 'Experience Score', 'Engagement Score', 'Satisfaction Score']]

Top 10 Satisfied Customers:


,MSISDN/Number,Experience Score,Engagement Score,Satisfaction Score
54715,3.503939e+10,1.134684,1.344089,1.239386
27546,3.503939e+10,1.150830,1.319259,1.235045
28333,3.503939e+10,1.163239,1.306276,1.234757
27591,3.503939e+10,1.142787,1.318609,1.230698
55849,3.503939e+10,1.175855,1.277841,1.226848
53085,3.503939e+10,1.189259,1.264333,1.226796
29957,3.503939e+10,1.150730,1.299990,1.225360
103447,3.503939e+10,1.193580,1.256433,1.225007
131840,3.503939e+10,1.163052,1.284688,1.223870
52837,3.503939e+10,1.126814,1.316208,1.221511


#### K = 2 Cluster on Engagment and Experiance Scores

##### k-means (k=2) on the engagement scores

In [54]:
# Select the 'Engagement Score' column
engagement_score = result_df_combined[['Engagement Score']]

# Perform k-means clustering with k=2
kmeans_engagement_score = KMeans(n_clusters=2, random_state=42)
result_df_combined['Engagement Cluster'] = kmeans_engagement_score.fit_predict(engagement_score)

# Display the results
print("User Data with Engagement Cluster (k=2):")
result_df_combined[['MSISDN/Number', 'Engagement Score', 'Engagement Cluster']].head(20)

c:\Users\User\miniconda3\envs\telecom_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


User Data with Engagement Cluster (k=2):


,MSISDN/Number,Engagement Score,Engagement Cluster
0,3.366496e+10,0.101995,0
1,3.368185e+10,0.465379,0
2,3.503939e+10,1.003227,1
3,3.503939e+10,1.198533,1
4,3.369980e+10,0.359548,0
5,3.366819e+10,0.548399,0
6,3.366537e+10,0.193915,0
7,3.503939e+10,1.166887,1
8,3.369874e+10,0.676446,1
9,3.365922e+10,0.643124,0


##### k-means (k=2) on the experiance scores

In [56]:
# Select the 'Experience Score' column
experience_score = result_df_combined[['Experience Score']]

# Perform k-means clustering with k=2
kmeans_experience_score = KMeans(n_clusters=2, random_state=42)
result_df_combined['Experience Cluster'] = kmeans_experience_score.fit_predict(experience_score)

# Display the results
print("User Data with Experience Cluster (k=2):")
result_df_combined[['MSISDN/Number', 'Experience Score', 'Experience Cluster']].head(20)

User Data with Experience Cluster (k=2):


c:\Users\User\miniconda3\envs\telecom_env\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,MSISDN/Number,Experience Score,Experience Cluster
0,3.366496e+10,1.080634,0
1,3.368185e+10,1.018734,0
2,3.503939e+10,0.986373,0
3,3.503939e+10,0.985932,0
4,3.369980e+10,0.986373,0
5,3.366819e+10,0.985631,0
6,3.366537e+10,0.982860,0
7,3.503939e+10,0.552276,1
8,3.369874e+10,0.985909,0
9,3.365922e+10,0.272044,1


Aggregate the average satisfaction & experience score per cluster. 

In [57]:
# ----- To Do --------- #

#### Export Satsfaction Table to Database

We will now export the our dataFrame that holds engagement, experience & satisfaction scores to our postgres database as a table. 

In [64]:
# Create satisfaction_result_df with selected columns
satisfaction_result_df = result_df_combined[['MSISDN/Number', 'Experience Score', 'Engagement Score', 'Satisfaction Score']].copy()

satisfaction_result_df.to_sql('satisfaction_result_df', con=engine, if_exists='replace', index=False)

1

Confirm satisfaction_result_df table is added to the data base:

In [66]:
# Establish a connection
with engine.connect() as connection:
    # Query the data from satisfaction_result_table and load it into a pandas DataFrame
    query = """
        SELECT *
        FROM satisfaction_result_df
    """
    satisfaction_result_query_df = pd.read_sql(query, connection)

satisfaction_result_query_df.head()

,MSISDN/Number,Experience Score,Engagement Score,Satisfaction Score
0,3.366496e+10,1.080634,0.101995,0.591314
1,3.368185e+10,1.018734,0.465379,0.742057
2,3.503939e+10,0.986373,1.003227,0.994800
3,3.503939e+10,0.985932,1.198533,1.092232
4,3.369980e+10,0.986373,0.359548,0.672960
